In [8]:
import numpy as np
import pandas as pd
import tensorflow as tf
from typing import Dict

from networks.recurrent import AutoRegressiveMultiStepLSTM

config: Dict = {
    "epochs": 2,
    "batch_size": 2,
    "learning_rate": 3e-4,
    "test_data_filepath": "/mnt/c/Users/JPhillips/ldz/data/processed/test.csv",
}

In [9]:
# Load data
test_df = pd.read_csv(config["test_data_filepath"])
test_dataset = tf.data.Dataset.from_tensor_slices(np.array(test_df))
# test_dataset = test_dataset.batch(1)

2022-05-17 17:44:10.063861: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: UNKNOWN ERROR (304)
2022-05-17 17:44:10.064040: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (NADT99): /proc/driver/nvidia/version does not exist
2022-05-17 17:44:10.064894: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [11]:
# model = tf.keras.load_model(cfg.predict.saved_model_path)
model = AutoRegressiveMultiStepLSTM(
    recurrent_units=256,
    num_features=66,   # Must match train_df.shape[1].
    num_out_steps=24,  # Must match config_data["window_label_width"].
)

In [13]:
model.compile(
    loss = tf.losses.MeanSquaredError(),
    optimizer = tf.keras.optimizers.Adam(learning_rate=5e-4),
)

In [19]:
test_dataset = test_dataset.batch(1)

In [21]:
example = next(iter(test_dataset))

In [22]:
example

<tf.Tensor: shape=(1, 72), dtype=float64, numpy=
array([[0.45936016, 0.36572433, 0.4548724 , 0.191038  , 0.47776333,
        0.45662473, 0.36224692, 0.46258746, 0.17518121, 0.76280574,
        0.44165699, 0.35646428, 0.3553595 , 0.13807089, 0.54663346,
        0.42051986, 0.34746386, 0.33176538, 0.13878284, 0.67548312,
        0.45936016, 0.36572433, 0.4548724 , 0.191038  , 0.47776333,
        0.4316278 , 0.35538444, 0.42616191, 0.15806331, 0.55658164,
        0.41527822, 0.34802442, 0.24700352, 0.08367039, 0.40055097,
        0.47622141, 0.36894862, 0.44583313, 0.1749343 , 0.17549296,
        0.47986177, 0.36943639, 0.41303588, 0.14510009, 0.23230139,
        0.49013648, 0.37132338, 0.43446622, 0.10837624, 0.53093145,
        0.44336365, 0.35684015, 0.46515668, 0.15707774, 0.78706977,
        0.45491264, 0.35939742, 0.38947688, 0.11980155, 0.75570968,
        0.45112997, 0.35600149, 0.40938574, 0.11746852, 0.73561293,
        0.        , 0.        , 1.        , 0.        , 1.        ,

In [18]:
model()

ValueError: Exception encountered when calling layer "rnn" (type RNN).

Shape (72, 1) must have rank at least 3

Call arguments received:
  • inputs=tf.Tensor(shape=(1, 72), dtype=float32)
  • mask=None
  • training=None
  • initial_state=None
  • constants=None

In [15]:
# model.built = True

saved_model_path = "/mnt/c/Users/JPhillips/ldz/outputs/2022-05-17/16-00-33/saved_models/checkpoints/AutoRegressiveMultiStepLSTM/saved_model.pb"
model.load_weights(saved_model_path)

2022-05-17 17:45:25.909480: W tensorflow/core/util/tensor_slice_reader.cc:96] Could not open /mnt/c/Users/JPhillips/ldz/outputs/2022-05-17/16-00-33/saved_models/checkpoints/AutoRegressiveMultiStepLSTM/saved_model.pb: DATA_LOSS: not an sstable (bad magic number): perhaps your file is in a different file format and you need to use a different restore operator?


ValueError: Unable to load weights saved in HDF5 format into a subclassed Model which has not created its variables yet. Call the Model first, then load the weights.

In [ ]:
predictions = model.predict(test_dataset)